In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import re
import babel
import subprocess
import os
import shutil
from prody import *

# 1- CLEANING INHIBITORS 

(USED R FOR THIS, SEE THE FILE cleaning_inhibitors.R to find the codes)

removed mutants, mutations and duplicates.

Input = inhibitors_from_chembl.csv
Output = Unique_Chembl.csv



# 2- Getting Ligand SMILES ID's

INPUT = Unique_Chembl.csv

OUTPUT = multiple smiles files, including all the ligands in Unique_Chembl.csv 

You can find already created files in /Important_Files/Ligand_smiles folder.
 

In [ ]:
chembl = pd.read_csv("Unique_Chembl.csv")
smilesids = chembl["Smiles"]
names = chembl["Molecule.ChEMBL.ID"]

In [ ]:
for i in range(0,len(smilesids)):                            
    file_name ='%s.smiles'% names[i]
    with open(file_name, 'w') as f:
        f.write(smilesids[i])

# 3- PROTEIN AND LIGAND PREPARE FOR DOCKING 

(USED GOOGLE COLAB AND BASH FOR THIS,CHECK OUT THE CODES IN Colab_Protein_Ligand_prepare_for_docking.ipynb)

preparing ligands

INPUT = Ligand smiles files inside Ligand_smiles folder

OUTPUT = Ligand mol2 and pdbqt files in Ligands file

preparing protein file

INPUT = alpha_templated.pdb file in Protein folder

OUTPUT = alpha_templated.pdbqt file in Protein folder

preparing substrates

INPUT = Substrate smiles files in Substrate folder

OUTPUT = Substrate pdbqt files in Substrate folder




# 4- VINA DOCKING FUNCTION 

There are multiple docking runs executed for this project.

1- Docking all ligands and substrates to a binding pocket that was suggested by EASYVS - ""-12.073,-0.049,5.027". The resulting files can be found under "Important Files/Docking Results/alpha_easyvs_pockets_run". The folders are named based on the exhaustiveness level and/or the day docking runs started and vina results are named based on the CHEMBL ID's.

INPUT: pdbqt files in Ligands, Substrates and Protein folder

OUTPUT: vina results under Important Files/Docking Results/alpha_easyvs_pockets_run

2- Docking all ligands and substrates to binding pockets generated by DogSiteScorer. The resulting files can be found under "Important Files/Docking Results/alpha_DogSite_new_pockets_run". The folders are named based on the pocket number (0 or 1) and the vina results are named based on the CHEMBL ID's.

INPUT: pdbqt files in Ligands, Substrates and Protein folder

OUTPUT: vina results under Important Files/Docking Results/alpha_DogSite_new_pockets_run

3- Docking function is used to dock Ciprofloxacin to all binding pockets obtained for each conformation extracted from MD simulations. The new conformation files are under "New_Conformations" folder. The files are named based on the date of Dynamics simulation and the number of frame which the structure was obtained. The results of this docking can be found under /Important Files/Docking Results/Ciprofloxacin_with_Coord_from_MD. The file coordinates.xlsx includes the necessary information for XYZ coordinates of each conformation and highly important for docking function.

INPUT: Ciprofloxacin pdbqt as substrate, coordinates.xlsx file for extracting coordinates of all conformations and norA structure pdb files under New_Conformations folder.

OUTPUT: vina results under Important Files/Docking Results/Ciprofloxacin_with_Coord_from_MD

4- Docking all ligands to all new nora conformations extracted from MD. Ligands are in Ligands folder, protein structures are in New Conformations folder. XYZ coordinates and radius of the binding pockets are in coordinates.xlsx which is critical for the docking process. Output files were under /Important_Files/Docking_results/All_Ligands_All_Conformations_Run. There were 267 ligands but at the end we could only get 194 molecules (due to time limit we stopped at this number), which reduced to 178 then 96 after filtering out the results. Folders are named based on the ligand name. and results are named based on the conformation and the pocket name.

INPUT: Ligands folder, New Conformations folder and coordinates.xlsx file.

OUTPUT: All_Ligands_All_Conformations_Run folder under /Important_Files/Docking_Results




In [ ]:
# DOCKING FUNCTION FOR FIRST ROUND

In [ ]:
alias vina /Users/esbusis/autodock_vina_1_1_2_mac_catalina_64bit/bin/vina #change this to exe directory

In [ ]:
# a is the center_x
# b is the center_y
# c is the center_z
# d is the size_x
# e is the size_y
#f is the size_z


def docking(protein,ligand,a,b,c,d,e,g,outfolder):
    
    matches = re.search(".(CHEMBL\d+).", ligand)
    ligname = matches.groups()[0]
    
    with open("config_singledock.txt","w") as f:
      f.write("#CONFIGURATION FILE (options not used are commented) \n")
      f.write("\n")
      f.write("#INPUT OPTIONS \n")
      f.write("receptor = %s \n" % str(protein))
      f.write("ligand = %s \n" % str(ligand))
      f.write("#flex = [flexible residues in receptor in pdbqt format] \n")
      f.write("#SEARCH SPACE CONFIGURATIONS \n")
      f.write("#Center of the box (values bxi, byi and bzi) \n")
#CHANGE THE FOLLOWING DATA WITH YOUR BOX CENTER COORDINATES  
      f.write("center_x = %d  \n" % float(a))
      f.write("center_y = %d  \n" % float(b))
      f.write("center_z = %d  \n" % float(c))
#CHANGE THE FOLLOWING DATA WITH YOUR BOX DIMENSIONS
      f.write("#Size of the box (values bxf, byf and bzf) \n")
      f.write("size_x = %d  \n" % float(d))
      f.write("size_y = %d  \n" % float(e))
      f.write("size_z = %d  \n" % float(g))
      f.write("#OUTPUT OPTIONS \n")
      f.write("out = %s \n" % (outfolder + "/" + str(ligname)+ "_docked.pdbqt"))
      f.write("log = %s \n" % (outfolder + "/" + str(ligname)+ "_docked.log"))
      f.write("\n")
      f.write("#OTHER OPTIONS \n")
      f.write("#cpu =  \n")
      f.write("exhaustiveness = 512")
      f.write("#num_modes = \n")
      f.write("#energy_range = \n")
      f.write("#seed = ")


#Executing AutoDock Vina with our configuration file
    %vina --config config_singledock.txt
    

    
    #!obabel -ipdbqt lig.pdbqt -opdb -O lig_dock.pdb -m
    
    #return lig_dock.pdb



In [ ]:
# DOCKING FOR FIRST ROUND

# DOCKING LIGANDS to a certain XYZ coordinate obtained from EASYVS

# Make sure the change directories of ligands, protein or substrates based on your project.

# The resulting files can be found under "Important Files/Docking Results/alpha_easyvs_pockets_run/ligands"

In [ ]:
nora = "/Protein/alpha_templated.pdbqt"
ligands = glob.glob("/Ligands/*pdbqt")


In [ ]:
os.mkdir("08_12_2021_Vina_Run")

for lig in ligands:
    docking(nora, lig,-12.073,-0.049,5.027,20,20,20,"08_12_2021_Vina_Run")

In [ ]:
# DOCKING FOR FIRST ROUND

# DOCKING SUBSTRATES to a certain XYZ coordinate obtained from EASYVS

# Make sure the change directories of ligands, protein or substrates based on your project.

# The resulting files can be found under "Important Files/Docking Results/alpha_easyvs_pockets_run/substrates"

In [ ]:
nora = "/Protein/alpha_templated.pdbqt"
substrates = glob.glob("/Substrates/*pdbqt")

In [ ]:
os.mkdir("Substrates_Run_512")

for lig in substrates:
    docking(nora, lig,-12.073,-0.049,5.027,20,20,20,"Substrates_Run_512")

In [ ]:
# DOCKING FUNCTION FOR SECOND ROUND

# Make sure the change directories of ligands, protein or substrates based on your project.

# The resulting files can be found under "Important Files/Docking Results/alpha_DogSite_new_pockets_run/ligands"
# and alpha_DogSite_new_pockets_run/substrates for the Substrates

In [ ]:
nora = "/Protein/alpha_templated.pdbqt"
ligands = glob.glob("/Ligands/*pdbqt")

In [ ]:
os.mkdir("27_02_2022_pocket_0_run_1")

for lig in ligands:
    docking(nora, lig,-4.04,-0.11,-2.87,20,20,20,"27_02_2022_pocket_0_run_1")

In [ ]:
nora = "/Protein/alpha_templated.pdbqt"
ligands = glob.glob("/Ligands/*pdbqt")

In [ ]:
os.mkdir("27_02_2022_pocket_0_run_1")

for lig in ligands:
    docking(nora, lig,-12.02,9.48,-0.80,20,20,20,"27_02_2022_pocket_0_run_1")

In [ ]:
# DOCKING FUNCTION FOR THIRD ROUND

# Here it is important to set the directory for ciprofloxacin and conformation files.

In [ ]:
coordinates = pd.read_excel("coordinates.xlsx")

In [ ]:
lig = '/Substrates/CHEMBL8_ciprofloxin.pdbqt'
os.mkdir("cipro_runs")
for i in range(len(coordinates)):
    docking(coordinates["File Name"][i],lig,coordinates["X Coordination"][i],coordinates["Y Coordination"][i],coordinates["Z Coordination"][i],coordinates["Radius"][i],coordinates["Radius"][i],coordinates["Radius"][i],"cipro_runs")

In [ ]:
# DOCKING FUNCTION FOR THE FOURTH ROUND

# Again, change the coordinates if necessary!

In [ ]:
coordinates = pd.read_excel("coordinates.xlsx")
ligands = glob.glob("Ligands/*pdbqt")

In [ ]:
for lig in ligands:
    matches = re.search(".(CHEMBL\d+).", lig)
    ligname = matches.groups()[0]
    a = "Run/" + ligname + "_run_1"
    os.mkdir(a)
    for i in range(len(coordinates)):
        docking("New_Coordinates/" + coordinates["File Name"][i],lig,coordinates["X Coordination"][i],coordinates["Y Coordination"][i],coordinates["Z Coordination"][i],coordinates["Radius"][i],coordinates["Radius"][i],coordinates["Radius"][i],a)

# 5- PARSING THE RESULTS OF DOCKING

The codes have multiple layers. 

1- Getting the metadata: The codes in here are used to obtain important information from Unique_Chembl.csv, because the original dataset is crowded and includes information that might not be usefull. These extracted values then saved to a csv named "metadata.csv".

INPUT: Unique_Chembl.csv

OUTPUT: metadata.csv

2- Extracting vina results: Extracting Binding affinity, RMSD scores from log files of ligand vina results obtained in the first round of docking and some important info as the Run, Ligand to create a new dataframe. This dataframe will be combined with metadata, hence we will obtain a dataframe containing metadata and vina results for each chemical.

For ligands in the first round

INPUT: Log files of docking results, under /Important_Files/Docking_Results/alpha_easyvs_pockets_run/ligands

OUTPUT: masterdf_alpha_easyvs_coordinates.csv

For substrates in the first round

INPUT: /Important_Files/Docking_Results/alpha_easyvs_pockets_run/substrates

OUTPUT: substrate_runs_alpha_easyvs.csv

For Ciprofloxacin in the third round

INPUT: /Important_Files/Docking_Results/Ciprofloxacin_with_Coord_from_MD

OUTPUT: cipro_new_conformations.csv

! The results for the second round docking has not been added here, because we did not use it in the study.

For all results in the fourth round:

INPUT: /Important_Files/Docking_Results/All_Ligands_All_Conformations_Run






## 5.1- Getting the metadata

In [ ]:
metadata = pd.read_csv("Unique_Chembl.csv")
metadata2 = metadata[["Molecule.ChEMBL.ID","AlogP","Standard.Type","Standard.Relation","Standard.Value","Comment","Assay.Description","Assay.ChEMBL.ID"]]
substrates = []
for i,j in enumerate(metadata2["Assay.Description"]):
    substrates.append([])
    if "ethidium bromide" in j:
        substrates[i].append("Ethidium Bromide")
    elif "ciprofloxacin" in j:
        substrates[i].append("Ciprofloxacin")
    elif "EtBr" in j:
        substrates[i].append("Ethidium Bromide")
    elif "Mg2+" in j:
        substrates[i].append("Hoechst 33342")
    elif "Berberine" in j:
        substrates[i].append("Berberine")
    else:
        substrates[i].append("None")
metadata2["Substrates"] = substrates
metadata2 = metadata2.rename(columns={"Molecule.ChEMBL.ID":"Ligand"})

metadata2.to_csv("metadata.csv")

## 5.2- Extracting Vina Results

Here, the results are extracted for Ligands docked in the first round.

In [ ]:
log_files = glob.glob("/Important_Files/Docking_Results/alpha_easyvs_pockets_run/ligands/*Vina_Run*/*log")

df3 = pd.DataFrame(columns=["Run","Ligand",'Pose','BA','RMSD','RMSD_2'])

for i in log_files:
        
    with open(i) as f:
        a = f.readlines()
    
    we = a[25:34]
    tt = []
    for k in we:
        r = k.split(" ")
        o = [float(el.rstrip("\n")) for el in r if el != ""]
        tt.append(o)

    er = i.split("/")
    ligname= er[9].rstrip("_docked.log")
    filename= er[8]

        
    df2 = pd.DataFrame(tt, columns=['Pose','BA','RMSD','RMSD_2'])
    df2['Pose','BA','RMSD','RMSD_2'] = tt
    df2["Ligand"] = ligname
    df2["Run"] = filename
    df2 = df2[["Run","Ligand","Pose","BA","RMSD","RMSD_2"]]
    
    df3 = df3.append(df2)
    
#df3.to_csv("df3.csv")
unique_runs = np.unique(df3["Run"].values)
unique_ligands = np.unique(df3["Ligand"].values)
df3 = df3.reset_index()
df3 = df3.drop("index", axis = 1)

for i in range(len(unique_runs)):
    Run = unique_runs[i]
    for j in range(len(unique_ligands)):
        Ligand = unique_ligands[j]
        
        df4 = df3[(df3["Run"] == Run) & (df3["Ligand"] == Ligand)]
        
        #print(df4)
        
        ind = df3[(df3["Run"] == Run) & (df3["Ligand"] == Ligand)].index
        
        #print(ind)
        
        std_of_ba = np.std(df4["BA"].values)
        
        df3.loc[ind, "Std_BA"] = std_of_ba

In [ ]:
masterdf = pd.merge(df3,metadata2, on= "Ligand")


Here, the results are extracted for Substrates docked in the first round.

In [ ]:
# For substrates

substrates_files = glob.glob("/Important_Files/Docking_Results/alpha_easyvs_pockets_run/substrates/*Substrates_Run*/*log")

df5 = pd.DataFrame(columns=["Run","Ligand",'Pose','BA','RMSD','RMSD_2'])

for i in substrates_files:
        
    with open(i) as f:
        a = f.readlines()
    
    we = a[25:26]
    tt = []
    for k in we:
        r = k.split(" ")
        o = [float(el.rstrip("\n")) for el in r if el != ""]
        tt.append(o)

    er = i.split("/")
    ligname= er[9].rstrip("_docked.log")
    filename= er[8]

        
    df2 = pd.DataFrame(tt, columns=['Pose','BA','RMSD','RMSD_2'])
    df2['Pose','BA','RMSD','RMSD_2'] = tt
    df2["Ligand"] = ligname
    df2["Run"] = filename
    df2 = df2[["Run","Ligand","Pose","BA","RMSD","RMSD_2"]]
    
    df5 = df5.append(df2)
    
df5.to_csv("substrate_runs_alpha_easyvs.csv")

Here, the results are extracted for Ciprofloxacin docked in the third round.

In [ ]:
# for alpha new pocket runs only pose 1 is collected
# change only file location inside glob.glob and the name of the output file

substrates_files = glob.glob("/Important_Files/Docking_Results/Ciprofloxacin_with_Coord_from_MD/*log")

df5 = pd.DataFrame(columns=["Run","Ligand",'Pose','BA','RMSD','RMSD_2'])

for i in substrates_files:
        
    with open(i) as f:
        a = f.readlines()
    
    we = a[25:26]
    tt = []
    for k in we:
        r = k.split(" ")
        o = [float(el.rstrip("\n")) for el in r if el != ""]
        tt.append(o)

    er = i.split("/")
    ligname= er[7].rstrip("_docked.log")
    filename= er[6]

        
    df2 = pd.DataFrame(tt, columns=['Pose','BA','RMSD','RMSD_2'])
    df2['Pose','BA','RMSD','RMSD_2'] = tt
    df2["Ligand"] = ligname
    df2["Run"] = filename
    df2 = df2[["Run","Ligand","Pose","BA","RMSD","RMSD_2"]]
    
    df5 = df5.append(df2)
    
df5.to_csv("cipro_new_conformations.csv")

Here, the results are extracted for fourth round. Not used in the final version but codes could come handy.

In [ ]:
log_files = glob.glob("/Important_Files/Docking_Results/All_Ligands_All_Conformations_Run/*_run_*/*log")

df3 = pd.DataFrame(columns=["Run","Ligand",'Pose','BA','RMSD','RMSD_2'])

for i in log_files:
        
    with open(i) as f:
        a = f.readlines()
    
    we = a[25:26]
    tt = []
    for k in we:
        r = k.split(" ")
        o = [float(el.rstrip("\n")) for el in r if el != ""]
        tt.append(o)

    er = i.split("\\")
    ligname= er[2].rstrip("_docked.log")
    filename= er[1]

        
    df2 = pd.DataFrame(tt, columns=['Pose','BA','RMSD','RMSD_2'])
    df2['Pose','BA','RMSD','RMSD_2'] = tt
    df2["Ligand"] = ligname
    df2["Run"] = filename
    df2 = df2[["Run","Ligand","Pose","BA","RMSD","RMSD_2"]]
    
    df3 = df3.append(df2)

# Adding Standard deviation of BA

#df3.to_csv("df3.csv")
unique_runs = np.unique(df3["Run"].values)
unique_ligands = np.unique(df3["Ligand"].values)
df3 = df3.reset_index()
df3 = df3.drop("index", axis = 1)

for i in range(len(unique_runs)):
    Run = unique_runs[i]
    for j in range(len(unique_ligands)):
        Ligand = unique_ligands[j]
        
        df4 = df3[(df3["Run"] == Run) & (df3["Ligand"] == Ligand)]
        
        #print(df4)
        
        ind = df3[(df3["Run"] == Run) & (df3["Ligand"] == Ligand)].index
        
        #print(ind)
        
        std_of_ba = np.std(df4["BA"].values)
        
        df3.loc[ind, "Std_BA"] = std_of_ba

# 6- BINDING SITE ANALYSYS


MAIN PURPOSES:

1- Get the first pose in ligand pdbqt files.

2- Convert these pdbqt files to pdb files using babel (need to be installed by sudo apt)

3- Combine new conformation pdb files with the ligand pdb files.

4- Use plip and get amino acid residues that are on the binding site for each docking results.

5- Parse xml files generated by plip analysis.


To re-accomplish these steps, the ligand docking results stored under All_Ligands_All_Conformations_Run can be used, with conformation files stored under New_Conformations. Here, you can find all the generated files under /Important_Files/PLIP_Results.




## 6.1- Getting the POSE 1 from output of vina - pdbqt file.

Here, the aim is to get only POSE 1 from docking results. The pdbqt files are parsed and new pdbqt files are generated including only pose_1 results.

Here we copied the docking results under Important_Files/Docking_Results/All_Ligands_All_Conformations_Run to PLIP_Results/Run_194 to save the files in another folder, in case we lose it during this new analysis


INPUT: All ligand pdbqt files in PLIP_Results/PLIP_Ligands

OUTPUT: _pose_1.pdbqt files within same folder

In [ ]:
def pose_1(input_file, output_file):
    
    with open(input_file,"r") as task:
        with open(output_file,"w") as output:
            
            for line in task:
                output.write(line)
                    
                if "ENDMDL" in line:
                    break


In [ ]:
files = glob.glob("/Important_Files/PLIP_Results/Run_194/*run*/*pdbqt")

In [ ]:
for i in files:
    pose_1(i,"{0}_pose_1.pdbqt".format(i))

In [ ]:
pose_files = glob.glob("/Important_Files/PLIP_Results/Run_194/*run*/*pose_1.pdbqt")

## 6.2- Preparing pdb files from pdbqt files by writing the commands to an sh file 

Remember, babel should be installed. But you can find bash files in the folder.

INPUT: _pose_1.pdbqt files in PLIP_Results/Ligands folder

OUTPUT: _pose_1.pdbqt.pdb files in the same folder.

First the pdb files for ligands are generated.
Then pdb files for new conformations are generated.


In [ ]:
file_name = "prepare_ligand_pdb.sh"
with open(file_name, 'w') as f:
    for i in pose_files:
        f.write("babel -ipdbqt {0} -opdb {1}.pdb ; \n".format(i,i))


!chmod +x prepare_ligand_pdb.sh
!bash prepare_ligand_pdb.sh


In [ ]:
coordinate_files = glob.glob("/New_Conformations/*pdbqt")

In [ ]:
file_name = "prepare_proteins_pdb.sh"
with open(file_name, 'w') as f:
    for i in coordinate_files:
        f.write("babel -ipdbqt {0} -opdb {1}.pdb ; \n".format(i,i))


!chmod +x prepare_proteins_pdb.sh
!bash prepare_proteins_pdb.sh

## 6.3- Combining PDB files of ligands and corresponding conformations.

This is done because PLIP requires a PDB file combining protein and ligand PDB files.

First we created a dataframe that contains ligand pdb files with corresponding protein pdb file.

Then we combined them by creating a function and using the function on all files.

INPUT: All protein pdb's, all ligand _pose_1.pdbqt.pdb's

OUTPUT: Combined pdb files. These files named based on the conformation, binding pocket on the conformation, ligand name and at the and there is a _combined.pdb tag. These files are available under PLIP_Results/Ligands

In [ ]:
protein_files = glob.glob("/New_Conformations/*pdb")
ligand_pose1_files = glob.glob("/Important_Files/PLIP_Results/Run_194/*run*/*pose_1.pdbqt.pdb")

In [ ]:
# Preparing a df that contains ligand pdb files with corresponding protein pdb file

oo = []
chembl = []

for i in ligand_pose1_files:
    a = i.split("/")
    vr = re.sub("_P_.*_docked.pdbqt_pose_1.pdbqt.pdb","", a[2])
    tr = vr.replace(".pdbqt", ".pdbqt.pdb")
    rt = "new_coord/" +tr
    
    oo.append(rt)    
    er = a[1].replace("_run_1","")
    chembl.append(er)

df = pd.DataFrame()

df["Ligands"] = ligand_pose1_files
df["Proteins"] = oo
df["Chembl"] = chembl


In [ ]:
def combining_pdbs_only_atoms(file1,file2,file3):
    
    data = data2 = ""

    with open(file1) as protein:
        for line in protein:
            if "ATOM" in line:
                data += line
    

    with open(file2) as pose:
        for line in pose:
            if "ATOM" in line:
                data2 += line
    
    data += "\n"
    data += data2

    with open (file3, 'w') as fp:
        fp.write(data)


In [ ]:
# Combining two pdb files.

for i in range(len(df)):
    combining_pdbs_only_atoms(df["Proteins"][i],df["Ligands"][i],"{0}_{1}_combined.pdb".format(df["Ligands"][i], df["Chembl"][i]))

## 6.4- Using PLIP

Here, first we get all combined files, then copy all to a new file, then prepare a bash script for using PLIP.

PLIP is also a CLI tool as babel and should be installed beforehand.



In [ ]:
combined_files = glob.glob("/Important_Files/PLIP_Results/Run_194/*run*/*combined.pdb")

In [ ]:
for file_name in combined_files:
    shutil.copy(file_name, "/Important_Files/PLIP_Results/PLIP_Ligands/")

In [ ]:
list_of_combined = glob.glob("/Important_Files/PLIP_Results/PLIP_Ligands/*pdb")

In [ ]:
file_name = "plip_for_combined.sh"
with open(file_name, 'w') as f:
    for i in list_of_combined:
        f.write("plipcmd -f {0} -x -o {1}_plip \n".format(i,i))


!chmod +x plip_for_combined.sh
!bash plip_for_combined.sh

## 6.5- Parsing PLIP results

PLIP generates XML files that includes binding sites for the given pdb files (that includes both protein and ligand).

First we extract information from these XML files, then we merge these information with metadata and vina results to obtain a master dataframe, which will be used for further analyses.

INPUT: PLIP xml results, Vina log files, metadata.csv

OUTPUT: masterdf.csv



In [ ]:
1# Parsing the xml report files

all_reports = glob.glob("/Important_Files/PLIP_Results/PLIP_Ligands/*plip/*xml")

In [ ]:
import xml.etree.ElementTree as ET

df3 = pd.DataFrame(columns = ["File Name","Interaction", "Residue Number", "Residue Type", "Residue Chain", "Ligand"])

for i in all_reports:
    
    mytree = ET.parse(i)
    myroot = mytree.getroot()

    r = i.split("/")
    t = r[1].split("_")
    str_match = [x for x in t if re.search('CHEMBL', x)]
    

    file_name = []
    interaction = []
    residue_number = []
    residue_type = []
    residue_chain = []
    ligand = []
    chembl_name = []

    for headers in myroot.findall("./bindingsite/interactions/"):
        for i in headers:
            for j in i.findall("restype_lIg"):
                if j.text.find("UNL") == 0 :

                    df = pd.DataFrame()
                    
                    
                    file_name.append(r[1])
                    chembl_name.append(str_match[0])
                    interaction.append(i.tag)
                    residue_number.append(i.find("resnr").text)
                    residue_type.append(i.find("restype").text)
                    residue_chain.append(i.find("reschaIn").text)
                    ligand.append(j.text)

    df = {"File Name": file_name, "Ligand": chembl_name ,"Interaction": interaction, "Residue Number": residue_number, "Residue Type" : residue_type, "Residue Chain" : residue_chain, "Residue Ligand" : ligand}

    df2 = pd.DataFrame(df)
    df3 = df3.append(df2)
    
df4 = df3.reset_index().drop("index",axis = 1).sort_values(by = ["Residue Number", "Residue Type"])

df4

In [ ]:
# Merging aminoacid data with metadata

metadata = pd.read_csv("metadata.csv")
proteins_and_metadata = pd.merge(df4,metadata, on = "Ligand").sort_values(by = ["Residue Number", "Residue Type"])


In [ ]:
log_files = glob.glob("/Important_Files/PLIP_Results/Run_194/*run*/*log")

run_results = pd.DataFrame(columns=["Pocket","Ligand",'Pose','BA','RMSD','RMSD_2'])

for i in log_files:
        
    with open(i) as f:
        a = f.readlines()
    
    we = a[25:26]
    tt = []
    for k in we:
        r = k.split(" ")
        o = [float(el.rstrip("\n")) for el in r if el != ""]
        tt.append(o)

    er = i.split("/")
    ligname= er[1].rstrip("_docked.log")
    filename= er[2]

        
    df2 = pd.DataFrame(tt, columns=['Pose','BA','RMSD','RMSD_2'])
    df2['Pose','BA','RMSD','RMSD_2'] = tt
    df2["Ligand"] = ligname
    df2["Pocket"] = filename
    df2 = df2[["Pocket","Ligand","Pose","BA","RMSD","RMSD_2"]]
    
    run_results = run_results.append(df2)

In [ ]:
# Cleaning the log file names -- will be appended to "run_results"

log_file_cleaned = []

a = run_results["Pocket"].to_list()

aa = run_results["Ligand"].to_list()

for i in range(len(run_results)):
    

    c = aa[i].split("_")

    chembl = c[0]

    d = a[i].split("_")

    
    if "12" in d:
    
        d.remove("coordinates.pdbqt")
        d.remove("docked.log")
        d.append(chembl)

        t = "_".join(d)
        log_file_cleaned.append(t)
        
        
    elif "09" in d:
    
        d.remove("coordinates")
        d.remove("docked.log")
        d[2] = d[2].replace(".pdbqt","")
        d.append(chembl)
        t = "_".join(d)
        
        log_file_cleaned.append(t)

run_results["Coordinates"] = log_file_cleaned

In [ ]:
# Cleaning the protein file names -- will be appended to "proteins_and_metadata"

proteins_and_metadata_file_name = proteins_and_metadata["File Name"].to_list()

protein_file_cleaned =[]

for i in range(len(proteins_and_metadata)):

    b = proteins_and_metadata_file_name[i].split("_")

    
    if "12" in b:
    
        b.remove("coordinates.pdbqt")
        b.remove("docked.pdbqt")
        b.remove("1.pdbqt.pdb")
        b.remove("combined.pdb")
        b.remove("plip")
        b.remove("pose")

        t = "_".join(b)
        
        protein_file_cleaned.append(t)
        
    elif "09" in b:
    
        b.remove("coordinates")
        b.remove("docked.pdbqt")
        b.remove("1.pdbqt.pdb")
        b.remove("combined.pdb")
        b.remove("plip")
        b.remove("pose")
        b[2] = b[2].replace(".pdbqt","")

        t = "_".join(b)
        
        protein_file_cleaned.append(t)
        
proteins_and_metadata["Coordinates"] = protein_file_cleaned

In [ ]:
# Merging aminoacid data with metadata

masterdf = pd.merge(run_results,proteins_and_metadata, on = "Coordinates").sort_values(by = ["Residue Number", "Residue Type"])

#masterdf.to_csv("masterdf.csv")

# Cipro PLIP Results Parsing

Parsing PLIP results for Ciprofloxacin.

The same procedures applies for cipro, but it is simpler due to size of the data.

INPUT: PLIP xml results for ciprofloxacin under cipro_combined.

OUTPUT: cipro_binding_sites.csv


In [ ]:
all_reports = glob.glob("/Important_Files/PLIP_Results/PLIP_Cipro/*plip/*xml")

In [ ]:
import xml.etree.ElementTree as ET

df9 = pd.DataFrame(columns = ["File Name","Interaction", "Residue Number", "Residue Type", "Residue Chain", "Ligand"])

for i in all_reports:
    
    mytree = ET.parse(i)
    myroot = mytree.getroot()

    r = i.split("/")
    t = r[1].split("_")


    file_name = []
    interaction = []
    residue_number = []
    residue_type = []
    residue_chain = []
    ligand = []
   

    for headers in myroot.findall("./bindingsite/interactions/"):
        for i in headers:
            for j in i.findall("restype_lIg"):
                if j.text.find("LIG") == 0 :

                    df7 = pd.DataFrame()
                    
                    
                    file_name.append(r[1])
                    interaction.append(i.tag)
                    residue_number.append(i.find("resnr").text)
                    residue_type.append(i.find("restype").text)
                    residue_chain.append(i.find("reschaIn").text)
                    ligand.append(j.text)

    df7 = {"File Name": file_name,"Interaction": interaction, "Residue Number": residue_number, "Residue Type" : residue_type, "Residue Chain" : residue_chain, "Residue Ligand" : ligand}

    df8 = pd.DataFrame(df7)
    df9 = df9.append(df8)
    
df10 = df9.reset_index().drop("index",axis = 1).sort_values(by = ["Residue Number", "Residue Type"])

df10

In [ ]:
df10.to_csv("cipro_binding_sites.csv")

# Binding Site Analysis for Cipro

INPUT: cipro_new_conformations.csv and cipro_binding_sites.csv

OUTPUT: cipro_bs_count.csv

In [ ]:
cipro = pd.read_csv("cipro_new_conformations.csv")
cipro_binding_sites = pd.read_csv("cipro_binding_sites.csv")

In [ ]:
proteins_and_metadata_file_name = cipro_binding_sites["File Name"].to_list()

protein_file_cleaned =[]

for i in range(len(cipro_binding_sites)):

    b = proteins_and_metadata_file_name[i].split("_")


    if "12" in b:

        b.remove("coordinates.pdbqt")
        b.remove("docked.pdbqt")
        b.remove("1.pdbqt.pdb")
        b.remove("combined.pdb")
        b.remove("plip")
        b.remove("pose")
        b.remove("cipro")

        t = "_".join(b)

        protein_file_cleaned.append(t)

    elif "09" in b:

        b.remove("coordinates")
        b.remove("docked.pdbqt")
        b.remove("1.pdbqt.pdb")
        b.remove("combined.pdb")
        b.remove("plip")
        b.remove("pose")
        b.remove("cipro")
        b[2] = b[2].replace(".pdbqt","")

        t = "_".join(b)

        protein_file_cleaned.append(t)

cipro_binding_sites["Coordinates"] = protein_file_cleaned

In [ ]:
proteins_and_metadata_file_name = cipro["Ligand"].to_list()

protein_file_cleaned =[]

for i in range(len(cipro)):

    b = proteins_and_metadata_file_name[i].split("_")


    if "12" in b:

        b.remove("coordinates.pdbqt")
        t = "_".join(b)

        protein_file_cleaned.append(t)

    elif "09" in b:

        b.remove("coordinates")

        b[2] = b[2].replace(".pdbqt","")

        t = "_".join(b)

        protein_file_cleaned.append(t)

protein_file_cleaned
cipro["Coordinates"] = protein_file_cleaned

In [ ]:
cipro_all = pd.merge(cipro,cipro_binding_sites, on = "Coordinates")


In [ ]:
cipro_all["Residue"] = cipro_all["Residue Number"].astype(str) +"-"+ cipro_all["Residue Type"]

In [ ]:
cipro_bs_count = cipro_all["Residue"].value_counts()
cipro_bs_count.to_csv("cipro_bs_count.csv")

# Analysis for Ligands

In here, we start to analyze the data we have in hand.

INPUT: masterdf.csv

OUTPUT: max_ba_df_194.csv and 178_ligands_in_study.csv



In [ ]:
unique_ligands = np.unique(masterdf["Ligand_y"])

max_ba_df = list()

for i in unique_ligands:
    
    df_lig = masterdf[masterdf['Ligand_y'] == i]
    ba_lig = np.unique(df_lig["BA"].values)
    max_ba_lig = np.min(ba_lig)
    df_lig_ba_max = df_lig[df_lig["BA"] == max_ba_lig]
    unique_coordinates = np.unique(df_lig_ba_max["Coordinates"].values)
    if len(unique_coordinates) > 1:
        #print(max_ba_lig, np.unique(df_lig_ba_max["Coordinates"].values))
        df_lig_ba_max = df_lig_ba_max[df_lig_ba_max["Coordinates"] == unique_coordinates[0]]
        print(max_ba_lig, np.unique(df_lig_ba_max["Coordinates"].values))
        
    max_ba_df.append(df_lig_ba_max)

In [ ]:
max_ba_df = pd.concat(max_ba_df)

In [ ]:
un_ligands = np.unique(max_ba_df["Ligand_y"])

for i in un_ligands:
    ligs = max_ba_df[max_ba_df["Ligand_y"] == i]
    print(len(np.unique(ligs["BA"].values)))

In [ ]:
max_ba_df.sort_values(by = "BA")

In [ ]:
# Using + operator to combine two columns

max_ba_df["Residue"] = max_ba_df['Residue Number'].astype(str) +"-"+ max_ba_df["Residue Type"]

In [ ]:
max_ba_df["Residue"].value_counts()

In [ ]:
max_ba_df.to_csv("max_ba_df_194.csv")

In [ ]:
chembl_ids = pd.read_csv("Unique_Chembl.csv")

In [ ]:
unique_ligs = np.unique(masterdf["Ligand_y"])

newdf = []

for i in unique_ligs:
    new = chembl_ids[chembl_ids["Molecule.ChEMBL.ID"] == i]
    newdf.append(new)

In [ ]:
newdf = pd.concat(newdf)

In [ ]:
newdf.to_csv("178_ligands_in_study.csv")

# Calculating Tanimoto score and Tanimoto Heatmap

INPUT: 178_ligands_in_study.csv # this file includes SMILES ID's which is used to calculate Tanimoto

OUTPUT: Cluster map of Tanimoto scores

In [ ]:
# calculating tanimoto

#!/usr/bin/env python3

import time
import random
import sys
from pathlib import Path
import seaborn as sns

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from rdkit import Chem
from rdkit import DataStructs
from rdkit.ML.Cluster import Butina
from rdkit.Chem import Draw
from rdkit.Chem import rdFingerprintGenerator
from rdkit.Chem.Draw import SimilarityMaps

# show full results
np.set_printoptions(threshold=sys.maxsize)


# Reading the input CSV file.

ligands_df = pd.read_csv("178_ligands_in_study.csv" , index_col=0 )
print(ligands_df.head())



# Creating molecules and storing in an array
molecules = []

"""Let's fetch the smiles from the input file and store in molecules array
        We have used '_' because we don't want any other column.
        If you want to fetch index and any other column, then replace '_' with 
            index and write column names after a ','.
"""

for _, smiles in ligands_df[[ "SMILES"]].itertuples():
    molecules.append((Chem.MolFromSmiles(smiles)))
molecules[:]



In [ ]:
similarities_arrays_df = pd.DataFrame(similarities, columns= ligands_df["Molecule.ChEMBL.ID"], index = ligands_df["Molecule.ChEMBL.ID"])
plot = sns.clustermap(similarities_arrays_df, cmap = "Blues")
#plot.savefig("tanimoto_similarities.png")

# Binding Site Heatmap

INPUT: max_ba_df_194.csv

OUTPUT: Binding site heatmap

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
max_ba_df_194 = pd.read_csv("max_ba_df_194.csv")

In [ ]:
max_ba_df_194 = max_ba_df_194[max_ba_df_194["Standard.Value"] > 20]

max_ba_df_194["Residue"].value_counts()

In [ ]:
binding_aa_nora = np.unique(max_ba_df_194["Residue Number"].sort_values())

unique_ligands = np.unique(max_ba_df_194["Ligand_y"])

all_arrays = []

for i in unique_ligands:
    
    ligs = max_ba_df_194[max_ba_df_194["Ligand_y"] == i]
    
    empty_array = np.zeros(len(binding_aa_nora)) 
    
    for j in range(len(binding_aa_nora)):
        
        aa = binding_aa_nora[j]
        
        if aa in ligs["Residue Number"].values:
            
            
            empty_array[j] = 1
            
    all_arrays.append(empty_array)

all_arrays_df = pd.DataFrame(all_arrays, columns= binding_aa_nora, index = unique_ligands)

plot = sns.clustermap(all_arrays_df, cmap="Blues", figsize= (20,20))

sns.set(font_scale = 5)

# Cluster Analysis for Binding Sites

INPUT: all_arrays_df from binding site analysis

OUTPUT: cluster_of_binding_sites.csv

In [ ]:
import scipy.cluster.hierarchy as sch

# retrieve clusters using fcluster 
d = sch.distance.pdist(all_arrays_df)
L = sch.linkage(d, method='complete')
# 0.2 can be modified to retrieve more stringent or relaxed clusters
clusters = sch.fcluster(L, 0.85*d.max(), 'distance')

# clusters indicices correspond to incides of original df

df_index = []
cluster_no = []

for i,cluster in enumerate(clusters):
    print(all_arrays_df.index[i], cluster)
    df_index.append(all_arrays_df.index[i])
    cluster_no.append(cluster)

In [ ]:
cluster_of_binding_sites = pd.DataFrame(list(zip(df_index,cluster_no)), columns= ["in","mol"])

In [ ]:
cluster_of_binding_sites.to_csv("cluster_of_binding_sites.csv")

# Cluster Analysis for Tanimoto Scores

INPUT: similarities_arrays_df from tanimoto score analysis

OUTPUT: cluster_of_similarity_scores.csv

In [ ]:
import scipy.cluster.hierarchy as sch

# retrieve clusters using fcluster 
d = sch.distance.pdist(similarities_arrays_df)
L = sch.linkage(d, method='complete')
# 0.2 can be modified to retrieve more stringent or relaxed clusters
clusters = sch.fcluster(L, 0.5*d.max(), 'distance')

# clusters indicices correspond to incides of original df

df_index = []
cluster_no = []

for i,cluster in enumerate(clusters):
    print(all_arrays_df.index[i], cluster)
    df_index.append(all_arrays_df.index[i])
    cluster_no.append(cluster)

In [ ]:
cluster_of_similarity_scores = pd.DataFrame(list(zip(df_index,cluster_no)), columns= ["in","mol"])

In [ ]:
cluster_of_similarity_scores.to_csv("cluster_of_similarity_scores.csv")

# Generating one file including also information of clusters

INPUT: 178_ligands_in_study.csv, clusterofsimilarityscores, clusterofbindingsites

OUTPUT: final_data_with_clusters.csv


In [ ]:
metadata = pd.read_csv("178_ligands_in_study.csv")

In [ ]:
clusterofsimilarityscores.columns = ["Index","Molecule.ChEMBL.ID", "Tanimoto Cluster"]

In [ ]:
clusterofbindingsites.columns = ["Index","Molecule.ChEMBL.ID", "Binding Site Cluster"]

In [ ]:
masterdf = pd.merge(metadata,clusterofsimilarityscores, on="Molecule.ChEMBL.ID")

In [ ]:
masterdf = pd.merge(masterdf,clusterofbindingsites, on="Molecule.ChEMBL.ID")

In [ ]:
tanimoto_cluster = masterdf["Tanimoto Cluster"]

In [ ]:
binding_site_cluster = masterdf["Binding Site Cluster"]

In [ ]:
masterdf.to_csv("final_data_with_clusters.csv")

# Confusion Matrix for Tanimoto vs Binding Sites

In [ ]:
import matplotlib.pyplot as plt
from sklearn import metrics
from matplotlib.pyplot import figure

confusion_matrix = metrics.confusion_matrix(tanimoto_cluster, binding_site_cluster)

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix)

cm_display.plot()
plt.show()

# Generating RMSD, RMSF plots


INPUT: DCD and PDB files under Important_Files/DCD_Files # DCD files are obtained by VMD and PDB files are the ones used as an input to MD.

OUTPUT: Plots of RMSD and RMSF



In [ ]:
file1 = parseDCD("Important_Files/DCD_Files/12_08_2021_10_skipped.dcd")

In [ ]:
structure1 = parsePDB("Important_Files/DCD_Files/12_08_step5_input.pdb")

In [ ]:
structure1 = structure1.select("protein")

In [ ]:
file1.setAtoms(structure1)

In [ ]:
file1 = file1[1:-1]

In [ ]:
file1.superpose()

In [ ]:
file1.setAtoms(structure1.calpha)

In [ ]:
rmsd_file1 = file1.getRMSDs()

In [ ]:
rmsf_file1 = file1.getRMSFs()

In [ ]:
fig = plt.figure(figsize=(10,8))
ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])
ax.plot(rmsd_file1,color = "black")
ax.set_xlabel("Time (ns)", fontsize = 15)
ax.set_ylabel("RMSD (Å)", fontsize = 15)
ax.set_title("RMSD over the simulation 08_12_2021", fontsize = 18)
ax.set_xticks([0,200,400,600,800,1000])
ax.set_xticklabels(['0','20','40','60',"80","100"])

In [ ]:
fig = plt.figure(figsize=(10,8))
ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])
ax.plot(rmsf_file1,color = "black")
ax.set_xlabel("Residue α Carbons", fontsize = 15)
ax.set_ylabel("RMSF (Å)", fontsize = 15)
ax.set_title("RMSF over the simulation 08_12_2021", fontsize =18)

In [ ]:
file2 = parseDCD("Important_Files/DCD_Files/21_09_2021.dcd")

In [ ]:
structure2 = parsePDB("Important_Files/DCD_Files/09_21_step5_input.pdb")

In [ ]:
structure2 = structure2.select("protein")

In [ ]:
file2.setAtoms(structure2)

In [ ]:
file2.setAtoms(structure2.calpha)

In [ ]:
file2 = file2[1:-1]

In [ ]:
file2.superpose()

In [ ]:
rmsd_file2 = file2.getRMSDs()

In [ ]:
rmsf_file2 = file2.getRMSFs()

In [ ]:
fig = plt.figure(figsize=(10,8))
ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])
ax.plot(rmsd_file2,color = "black")
ax.set_xlabel("Time (ns)", fontsize = 15)
ax.set_ylabel("RMSD (Å)", fontsize = 15)
ax.set_title("RMSD over the simulation 09_21_2021", fontsize = 18)
ax.set_xticks([0,200,400,600,800,1000])
ax.set_xticklabels(['0','20','40','60',"80","100"])


In [ ]:
plt.figure(figsize = (10,8))
plt.plot(rmsf_file2, color = "black")
plt.xlabel("Residue α Carbons",fontsize = 15)
plt.ylabel("RMSF (Å)",fontsize = 15)
plt.title("RMSF over the simulation 09_21_2021",fontsize = 18)